In [ ]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim import Adam
import torch.nn.functional as F

import csv
from skimage import io

from PIL import Image
import pandas as pd

import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

import matplotlib.pyplot as plt
import time
import os
import copy

verbose = False
USE_CUDA = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

import import_ipynb
import CapsNet_Layers 

def lr_decrease(optimizer, lr_clip):  
    for param_group in optimizer.param_groups:
        init_lr = param_group['lr'] 
        param_group['lr'] = init_lr*lr_clip
        
def isnan(x):
    return x != x   

dataset_transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),        
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])


batch_size = 32
NUM_CLASSES = 10
print("CIFAR10")
image_datasets = {'train': datasets.CIFAR10('../data', train=True, download=True, transform=dataset_transform),'val': datasets.CIFAR10('../data', train=False, download=True, transform=dataset_transform)}
print("Initializing Datasets and Dataloaders...")
dataloaders = {'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True) , 'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=True) }
print("Initializing Datasets and Dataloaders...")

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [ ]:
print("=> using model CapsuleNET")
model = CapsNet_Layers.CapsNet(NUM_CLASSES)
model = model.to(device)

if USE_CUDA:
    model = model.to(device)#cuda()
    print('cuda')
optimizer = Adam(model.parameters(),lr = 0.0001)
criterion = nn.CrossEntropyLoss()

print(torch.version)

n_epochs = 10
x = range(0,n_epochs)
accuracy_train = []
loss_train = []

start = time.time()

for epoch in range(n_epochs): 
    capsule_net.train() 
    train_loss = 0 
    train_accuracy = 0
    
    batch_accuracy = []

    print('epoch {}:{}'.format(epoch+1, n_epochs)) 
    for batch_id, (data, target) in enumerate(dataloaders['train']):
        target =torch.eye(NUM_CLASSES).index_select(dim=0, index=target)
        data, target = Variable(data), Variable(target)

        if USE_CUDA:
            data, target = data.to(device), target.to(device)#.cuda()

        optimizer.zero_grad()
        output, reconstructions, masked = capsule_net(data)
        loss = capsule_net.loss(data, output, target, reconstructions)
        
        if isnan(loss):
            Data_name_file = "data_eq"+str(epoch)+"_batch_id"+str(batch_id)+".pt"
            Output_name_file = "output_eq"+str(epoch)+"_batch_id"+str(batch_id)+".pt"
            break
            
        loss.backward()
        optimizer.step()

        train_loss += loss.data[0]
        train_accuracy += (sum(np.argmax(masked.data.cpu().numpy(), 1) == np.argmax(target.data.cpu().numpy(), 1)) / float(batch_size))
        

        if batch_id % 100 == 0:
            print("train accuracy:", sum(np.argmax(masked.data.cpu().numpy(), 1) == 
                                   np.argmax(target.data.cpu().numpy(), 1)) / float(batch_size))
            print("loss {}".format(loss.data[0]))
            batch_accuracy.append(sum(np.argmax(masked.data.cpu().numpy(), 1) == 
                                   np.argmax(target.data.cpu().numpy(), 1)) / float(batch_size))
    accuracy_train.append(np.mean(batch_accuracy))
    del batch_accuracy
    loss_train.append(train_loss/len(dataloaders['train']))
    
end = time.time()
print("Training time execution {}".format(end-start))
print("Loss value for test phase: {}".format(train_loss /  len(dataloaders['train'])))
print("Accuracy value for test phase: {}".format(train_accuracy /  len(dataloaders['train'])))